In [279]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split

from pgmpy.estimators import HillClimbSearch, ExhaustiveSearch, BayesianEstimator
from pgmpy.estimators import ConstraintBasedEstimator, K2Score, BicScore, BDeuScore
from pgmpy.estimators import MaximumLikelihoodEstimator

from pgmpy.models import BayesianModel


In [287]:
septicemia2017 = pd.read_csv('../data/sparcs/septicemia/summary_2017.csv')
septicemia2017.head()

,apr_drg_code,facility_name,payment_typology_1,apr_severity_of_illness,patients,mean_length_of_stay,mean_total_charges_day,System,area_sqmi,Number of Beds
0,720,Bellevue Hospital Center,Blue Cross/Blue Shield,Major,4,14.000000,6541.667500,Health+,0.77,912
1,720,Bellevue Hospital Center,Blue Cross/Blue Shield,Minor,1,3.000000,6103.620000,Health+,0.77,912
2,720,Bellevue Hospital Center,Blue Cross/Blue Shield,Moderate,7,4.857143,6945.730000,Health+,0.77,912
3,720,Bellevue Hospital Center,Medicare,Extreme,45,13.133333,7090.089778,Health+,0.77,912
4,720,Bellevue Hospital Center,Medicare,Major,76,10.039474,6249.888289,Health+,0.77,912


In [332]:
septicemia2017 = pd.read_csv('../data/sparcs/heart_failure/summary_2017.csv')
septicemia2017.head()

,apr_drg_code,facility_name,payment_typology_1,apr_severity_of_illness,patients,mean_length_of_stay,mean_total_charges_day,System,area_sqmi,Number of Beds
0,194,Bellevue Hospital Center,Blue Cross/Blue Shield,Major,1,4.000000,7072.220000,Health+,0.77,912
1,194,Bellevue Hospital Center,Blue Cross/Blue Shield,Moderate,3,4.666667,6670.496667,Health+,0.77,912
2,194,Bellevue Hospital Center,Medicare,Extreme,11,10.909091,6845.906364,Health+,0.77,912
3,194,Bellevue Hospital Center,Medicare,Major,54,10.518519,5975.812407,Health+,0.77,912
4,194,Bellevue Hospital Center,Medicare,Minor,14,3.214286,7439.227857,Health+,0.77,912


In [333]:
def flatten_categories(df, col):
    new_cols = []
    for category in df[col].unique():
        new_cols.append('is_' + str(category))
        df['is_' + str(category)] = df.apply(lambda x: 1 if x[col] == category else 0, axis=1)
    return (df,new_cols)


septicemia2017, payment_cols = flatten_categories(septicemia2017, 'payment_typology_1')
septicemia2017, severe_cols  = flatten_categories(septicemia2017, 'apr_severity_of_illness')
septicemia2017, system_cols  = flatten_categories(septicemia2017, 'System')

In [341]:
cols = ['patients', 'mean_length_of_stay','mean_total_charges_day']#, 'Number of Beds' ]

data = septicemia2017[cols+payment_cols+severe_cols+system_cols].copy()
data.head()

,patients,mean_length_of_stay,mean_total_charges_day,is_Blue Cross/Blue Shield,is_Medicare,is_Private Health Insurance,is_Self-Pay,is_Major,is_Moderate,is_Extreme,is_Minor,is_Health+,is_Mount Sinai,is_Others,is_SUNY,is_Northwell,is_Montefiore,is_NewYork-Presbyterian,is_NYU Langone
0,1,4.000000,7072.220000,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
1,3,4.666667,6670.496667,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
2,11,10.909091,6845.906364,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0
3,54,10.518519,5975.812407,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0
4,14,3.214286,7439.227857,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0


In [342]:
for i in data.iloc[:,:]:
    data[i] = pd.cut(data[i], bins=10, labels=False)
data.head()

,patients,mean_length_of_stay,mean_total_charges_day,is_Blue Cross/Blue Shield,is_Medicare,is_Private Health Insurance,is_Self-Pay,is_Major,is_Moderate,is_Extreme,is_Minor,is_Health+,is_Mount Sinai,is_Others,is_SUNY,is_Northwell,is_Montefiore,is_NewYork-Presbyterian,is_NYU Langone
0,0,0,2,9,0,0,0,9,0,0,0,9,0,0,0,0,0,0,0
1,0,0,1,9,0,0,0,0,9,0,0,9,0,0,0,0,0,0,0
2,0,2,2,0,9,0,0,0,0,9,0,9,0,0,0,0,0,0,0
3,1,2,1,0,9,0,0,9,0,0,0,9,0,0,0,0,0,0,0
4,0,0,2,0,9,0,0,0,0,0,9,9,0,0,0,0,0,0,0


In [343]:
hc = HillClimbSearch(data, scoring_method = BicScore(data))
bic_best_model = hc.estimate()
print(bic_best_model.edges(), "\n")

[('mean_length_of_stay', 'is_Minor'), ('mean_total_charges_day', 'is_Health+'), ('mean_total_charges_day', 'is_Montefiore'), ('mean_total_charges_day', 'is_Others'), ('mean_total_charges_day', 'is_NYU Langone'), ('is_Blue Cross/Blue Shield', 'is_Medicare'), ('is_Blue Cross/Blue Shield', 'is_Private Health Insurance'), ('is_Blue Cross/Blue Shield', 'is_Self-Pay'), ('is_Medicare', 'patients'), ('is_Medicare', 'is_Extreme'), ('is_Private Health Insurance', 'is_Medicare'), ('is_Self-Pay', 'is_Medicare'), ('is_Self-Pay', 'is_Private Health Insurance'), ('is_Major', 'is_Moderate'), ('is_Extreme', 'is_Moderate'), ('is_Extreme', 'is_Major'), ('is_Extreme', 'mean_length_of_stay'), ('is_Minor', 'is_Moderate'), ('is_Minor', 'is_Major'), ('is_Health+', 'is_Mount Sinai'), ('is_Health+', 'is_Northwell'), ('is_Health+', 'is_Others'), ('is_Health+', 'is_SUNY'), ('is_Health+', 'is_NewYork-Presbyterian'), ('is_Mount Sinai', 'is_SUNY'), ('is_Mount Sinai', 'is_Northwell'), ('is_Mount Sinai', 'is_NewYork-P

In [344]:
def LL(x,model,verbose=False):
    loglike = 0
    for cpd in model.get_cpds():
        temp_cpd = cpd.copy()
        thevariable = temp_cpd.variable
        theparents = model.predecessors(thevariable)
        for parent in theparents:
            temp_cpd.reduce([(parent, x[parent])])
#         print(temp_cpd.get_values())
#         print(x[thevariable])
        if x[thevariable] < len(temp_cpd.get_values()): # I added this to stop it from failing
            theprob = temp_cpd.get_values()[x[thevariable],0]
            if verbose:
                print (thevariable,theparents,theprob)
            loglike += np.log(theprob)
    return loglike

In [345]:
model = BayesianModel( bic_best_model.edges() )
model.fit(data, estimator=MaximumLikelihoodEstimator)
exmp = data.apply(lambda x: LL(x, model), axis=1)
exmp2=pd.Series(exmp)
exmp2.index = septicemia2017.index
exmp2.sort_values().head(5)

C:\Users\erikl\Anaconda2\envs\AppliedDataScience\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app


49          -inf
50    -11.614074
52    -11.481226
486   -10.830959
281   -10.487455
dtype: float64

In [346]:
print("Top Anomalies")
septicemia2017.iloc[exmp2.sort_values().head(5).index]

Top Anomalies


,apr_drg_code,facility_name,payment_typology_1,apr_severity_of_illness,patients,mean_length_of_stay,mean_total_charges_day,System,area_sqmi,Number of Beds,...,is_Extreme,is_Minor,is_Health+,is_Mount Sinai,is_Others,is_SUNY,is_Northwell,is_Montefiore,is_NewYork-Presbyterian,is_NYU Langone
49,194,Calvary Hospital Inc,Medicare,Extreme,10,35.100000,2073.672000,Others,4.495,200,...,1,0,0,0,1,0,0,0,0,0
50,194,Calvary Hospital Inc,Medicare,Major,55,20.418182,2146.573455,Others,4.495,200,...,0,0,0,0,1,0,0,0,0,0
52,194,Calvary Hospital Inc,Medicare,Moderate,59,16.694915,2121.762034,Others,4.495,200,...,0,0,0,0,1,0,0,0,0,0
486,194,University Hospital of Brooklyn,Medicare,Major,116,10.275862,5586.447672,SUNY,2.265,342,...,0,0,0,0,0,1,0,0,0,0
281,194,Montefiore Medical Center - Henry & Lucy Moses...,Private Health Insurance,Moderate,54,5.185185,13812.416111,Montefiore,1.721,816,...,0,0,0,0,0,0,0,1,0,0


In [347]:
print("Least Anomalous")
septicemia2017.iloc[exmp2.sort_values(ascending=False).head(5).index] #11483.51,4901.12,15590.689779

Least Anomalous


,apr_drg_code,facility_name,payment_typology_1,apr_severity_of_illness,patients,mean_length_of_stay,mean_total_charges_day,System,area_sqmi,Number of Beds,...,is_Extreme,is_Minor,is_Health+,is_Mount Sinai,is_Others,is_SUNY,is_Northwell,is_Montefiore,is_NewYork-Presbyterian,is_NYU Langone
73,194,Elmhurst Hospital Center,Blue Cross/Blue Shield,Minor,3,4.0,7027.393333,Health+,10.090,545,...,0,1,1,0,0,0,0,0,0,0
382,194,North Central Bronx Hospital,Blue Cross/Blue Shield,Minor,1,2.0,7953.730000,Health+,3.220,213,...,0,1,1,0,0,0,0,0,0,0
121,194,Jacobi Medical Center,Blue Cross/Blue Shield,Minor,1,1.0,8156.080000,Health+,7.016,457,...,0,1,1,0,0,0,0,0,0,0
59,194,Coney Island Hospital,Blue Cross/Blue Shield,Minor,2,2.0,7876.005000,Health+,9.415,371,...,0,1,1,0,0,0,0,0,0,0
183,194,Lincoln Medical & Mental Health Center,Blue Cross/Blue Shield,Minor,2,1.0,7764.400000,Health+,2.933,362,...,0,1,1,0,0,0,0,0,0,0
